In [1]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime

In [2]:
ee.Initialize()

# VETTORI
countries = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')
wsheds = ee.FeatureCollection('ft:1IXfrLpTHX4dtdj1LcNXjJADBB-d93rkdJ9acSEWK')

In [3]:
# AREA OF INTEREST
region = [[-25.0, -40.0], [65.0, -40.0], [65.0, 40.0], [-30.0, 40.0], [-30.0, -40.0]]

In [4]:
wpbm_stats = ee.Image('projects/fao-wapor/L1-WPbmY2015-sample');

In [5]:
def vizualize_image(wbpm,regione_plot,scala):    
    visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region": regione_plot,
             "scale" : scala}
    
    url_WPbm = wbpm.getThumbUrl(visparams)
    return url_WPbm

In [6]:
%matplotlib inline  

In [7]:
Image(url=vizualize_image(wpbm_stats,region,250))


# Statistiche WPbm per paese e anno

In [8]:
largeSheds = wsheds.filter(ee.Filter.gt('AreaSqKm', 25000))
'Count after filtering by size:', largeSheds.size().getInfo()

('Count after filtering by size:', 167)

In [9]:
nomeCountry = 'Sudan'
justCountry = countries.filter(ee.Filter.eq('Country', nomeCountry))
cutPoly = justCountry.geometry() 
cutBoundingBox = cutPoly.bounds(1)

country_mean = wpbm_stats.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = cutPoly,
    scale = 250,
    maxPixels = 1e9
)
mean = country_mean.getInfo()
mean['mean'] = mean.pop('b1')

reducers = ee.Reducer.minMax().combine(
  reducer2 = ee.Reducer.stdDev(),
  sharedInputs = True
)

# Use the combined reducer to get the min max and SD of the image.
stats = wpbm_stats.reduceRegion(
  reducer = reducers,
  bestEffort = True,
  geometry = cutPoly,
  scale = 250,
)

# Display the dictionary of band means and SDs.
min_max_std = stats.getInfo()
min_max_std['min'] = min_max_std.pop('b1_min')
min_max_std['std'] = min_max_std.pop('b1_stdDev')
min_max_std['max'] = min_max_std.pop('b1_max')
min_max_std.update(mean)
min_max_std

{'max': 3.7759787530379403,
 'mean': 0.3441474380191768,
 'min': 0.0,
 'std': 0.23828625422463104}

## statistiche multipoligono

In [119]:
# VETTORI 1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ
countries_gaul = ee.FeatureCollection('ft:1ZDEMjtnWm_smu7l_z3fx91BbxyCRzP2A3cEMrEiP')
#countries_gaul = ee.FeatureCollection('ft:1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ')
africa_bbox = ee.Geometry.Rectangle(-15.64, -33.58, 59.06, 25.96)
filtered = countries_gaul.filterBounds(africa_bbox)

In [132]:
country_number = filtered.size().getInfo()
country_number

69

## Means

In [121]:
means = wpbm_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = 250,
)

## Min Max

In [122]:
minMaxs = wpbm_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.minMax(),
    scale = 250
)

## MinMax StdDev combined

In [123]:
reducers = ee.Reducer.minMax().combine(
    reducer2 = ee.Reducer.stdDev(),
    sharedInputs = True
);

minMaxStds = wpbm_stats.reduceRegions(
            collection = filtered,
            reducer = reducers,
            scale = 250,
);

In [124]:
features_m = means.getInfo()['features']

In [125]:
features_m[6]['properties']

{u'bbox': u'27.0290699005127,-30.6689605712891,29.4657611846924,-28.5720596313477',
 u'bbox_flag': 1.0,
 u'bbox_lod': u'27.0290699005127,-30.6689605712891,29.4657611846924,-28.5720596313477',
 u'claimed_by': u'',
 u'claimed_by_ft_style': None,
 u'fid': 140.0,
 u'gaul_code': 142.0,
 u'geometry_vertex_count': 1295.0,
 u'import_notes': u'',
 u'iso3': u'LSO',
 u'mean': 1.9201431284795158,
 u'name': u'Lesotho',
 u'region': u'Africa',
 u'region_ft_style': 0.0,
 u'status': u'Member State',
 u'status_ft_style': 0.0,
 u'subregion': u'Southern Africa',
 u'subregion_ft_style': 3.0}

In [126]:
features_m[6]['properties']['name']

u'Lesotho'

In [127]:
features_mms = minMaxStds.getInfo()['features']

In [128]:
features_mms[22]['properties']

{u'bbox': u'19.9993648529053,-26.9072513580322,29.3607807159424,-17.7788066864014',
 u'bbox_flag': 1.0,
 u'bbox_lod': u'19.9993648529053,-26.9072513580322,29.3607807159424,-17.7788066864014',
 u'claimed_by': u'',
 u'claimed_by_ft_style': None,
 u'fid': 35.0,
 u'gaul_code': 35.0,
 u'geometry_vertex_count': 3975.0,
 u'import_notes': u'',
 u'iso3': u'BWA',
 u'max': 3.6671032790041074,
 u'min': 0.0010593921454182481,
 u'name': u'Botswana',
 u'region': u'Africa',
 u'region_ft_style': 0.0,
 u'status': u'Member State',
 u'status_ft_style': 0.0,
 u'stdDev': 0.19113759179866738,
 u'subregion': u'Southern Africa',
 u'subregion_ft_style': 3.0}

In [129]:
features_mms[22]['properties']['name']

u'Botswana'

for (var i = 0; i < features_m.length; ++i) {
  var feat_m = features_m[i].properties;
  var feat_mms = features_mms[i].properties;
  print(feat_m.fid + "," + feat_m.gaul_code + "," + feat_m.iso3 + "," + 
        feat_m.name + "," + feat_m.mean + ',' + feat_mms.min + ',' + 
        feat_mms.max + ',' + feat_mms.stdDev);
}

means_data_by_country['features'][12]['properties']
numero_record = features_m.size().getInfo()

In [133]:
class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)() # retain local pointer to value
        return value                     # faster to return than dict lookup

In [134]:
stats_cntry = Vividict()
#d['foo']['bar']
#d['foo']['baz']
#d['fizz']['buzz']
#d['primary']['secondary']['tertiary']['quaternary']
#pprint.pprint(d)

In [135]:
import pprint

In [136]:
for i in range(country_number):
    #print i,features_m[i]['properties']['name']
    stats_cntry[i]['name'] = features_m[i]['properties']['name']
    stats_cntry[i]['iso3'] = features_m[i]['properties']['iso3']
    stats_cntry[i]['fid'] = features_m[i]['properties']['fid']
    stats_cntry[i]['mean'] = features_m[i]['properties']['mean']
    stats_cntry[i]['gaul_code'] = features_m[i]['properties']['gaul_code']        
    stats_cntry[i]['min'] = features_mms[i]['properties']['min']
    stats_cntry[i]['max'] = features_mms[i]['properties']['max']  
    stats_cntry[i]['stdDev'] = features_mms[i]['properties']['stdDev']

0 Zambia
1 Zimbabwe
2 Kuwait
3 Algeria
4 Sudan
5 Hala'ib triangle
6 Lesotho
7 Angola
8 Egypt
9 Liberia
10 Libya
11 Ilemi triangle
12 Kafia Kenji
13 Kaka
14 Madagascar
15 Malawi
16 Mali
17 Bahrain
18 Mauritania
19 Benin
20 Morocco
21 Mozambique
22 Botswana
23 Namibia
24 Burkina Faso
25 Burundi
26 Cameroon
27 Niger
28 Nigeria
29 Central African Republic
30 Chad
31 Oman
32 Comoros
33 Congo
34 Qatar
35 CA'te d'Ivoire
36 Democratic Republic of the Congo
37 Djibouti
38 Rwanda
39 South Sudan
40 Equatorial Guinea
41 Eritrea
42 Sao Tome and Principe
43 Ethiopia
44 Saudi Arabia
45 Senegal
46 Seychelles
47 Sierra Leone
48 Gabon
49 Somalia
50 Gambia
51 South Africa
52 Ghana
53 Swaziland
54 Abyei
55 Guinea-Bissau
56 Guinea
57 Togo
58 Tunisia
59 Iran (Islamic Republic of)
60 Uganda
61 Iraq
62 United Arab Emirates
63 Israel
64 United Republic of Tanzania
65 Jordan
66 Kenya
67 Western Sahara
68 Yemen


In [137]:
pprint.pprint(stats_cntry)

{0: {'fid': 265.0,
     'gaul_code': 270.0,
     'iso3': u'ZMB',
     'max': 6.420362077096733,
     'mean': 1.4678584836080688,
     'min': 0.010967265670640831,
     'name': u'Zambia',
     'stdDev': 0.5367549088306263},
 1: {'fid': 266.0,
     'gaul_code': 271.0,
     'iso3': u'ZWE',
     'max': 8.52988423640644,
     'mean': 1.1081521754210037,
     'min': 0.0029675901075164996,
     'name': u'Zimbabwe',
     'stdDev': 0.48243519017253184},
 2: {'fid': 135.0,
     'gaul_code': 137.0,
     'iso3': u'KWT',
     'max': 1.4625105501211912,
     'mean': 0.04066641882164415,
     'min': 0.0,
     'name': u'Kuwait',
     'stdDev': 0.06915770876719184},
 3: {'fid': 4.0,
     'gaul_code': 4.0,
     'iso3': u'DZA',
     'max': 9.079370658862421,
     'mean': 0.6883704588119687,
     'min': 0.0,
     'name': u'Algeria',
     'stdDev': 0.46109457312340096},
 4: {'fid': 6.0,
     'gaul_code': 6.0,
     'iso3': u'SDN',
     'max': 3.0354629539074005,
     'mean': 0.2330979292241856,
     'min': 

In [138]:
for key,value in stats_cntry.items():
    print value

{'name': u'Zambia', 'stdDev': 0.5367549088306263, 'min': 0.010967265670640831, 'max': 6.420362077096733, 'iso3': u'ZMB', 'fid': 265.0, 'gaul_code': 270.0, 'mean': 1.4678584836080688}
{'name': u'Zimbabwe', 'stdDev': 0.48243519017253184, 'min': 0.0029675901075164996, 'max': 8.52988423640644, 'iso3': u'ZWE', 'fid': 266.0, 'gaul_code': 271.0, 'mean': 1.1081521754210037}
{'name': u'Kuwait', 'stdDev': 0.06915770876719184, 'min': 0.0, 'max': 1.4625105501211912, 'iso3': u'KWT', 'fid': 135.0, 'gaul_code': 137.0, 'mean': 0.04066641882164415}
{'name': u'Algeria', 'stdDev': 0.46109457312340096, 'min': 0.0, 'max': 9.079370658862421, 'iso3': u'DZA', 'fid': 4.0, 'gaul_code': 4.0, 'mean': 0.6883704588119687}
{'name': u'Sudan', 'stdDev': 0.13511698501839978, 'min': 0.0, 'max': 3.0354629539074005, 'iso3': u'SDN', 'fid': 6.0, 'gaul_code': 6.0, 'mean': 0.2330979292241856}
{'name': u"Hala'ib triangle", 'stdDev': 0.042389375957820115, 'min': 0.0, 'max': 0.41907272635730786, 'iso3': u'', 'fid': 270.0, 'gaul_

In [139]:
import pandas as pd

In [140]:
stats_cntry_df = pd.DataFrame.from_dict(stats_cntry)

In [142]:
print stats_cntry_df.head()

                0        1          2        3         4         5        6   \
fid            265      266        135        4         6       270      140   
gaul_code      270      271        137        4         6     40760      142   
iso3           ZMB      ZWE        KWT      DZA       SDN                LSO   
max        6.42036  8.52988    1.46251  9.07937   3.03546  0.419073  28.5681   
mean       1.46786  1.10815  0.0406664  0.68837  0.233098  0.065431  1.92014   

                7         8         9     ...           59       60        61  \
fid              8       272       141    ...          117      248       118   
gaul_code        8     40765       144    ...          117      253       118   
iso3           AGO       EGY       LBR    ...          IRN      UGA       IRQ   
max        7.56753   5.25782   3.10682    ...      78.2276  45.1061   3.08155   
mean       0.98743  0.377685  0.975488    ...     0.734687   1.0069  0.319032   

                  62        63  

In [194]:
means_df = pd.DataFrame.from_dict(features_m)

In [197]:
df = pd.DataFrame(data=features_m[1:], columns = features_m[0])
df

,geometry,type,id,properties
0,"{u'type': u'Polygon', u'geodesic': True, u'coo...",Feature,2,"{u'status': u'Member State', u'name': u'Zimbab..."
1,"{u'type': u'MultiPolygon', u'geodesic': True, ...",Feature,3,"{u'status': u'Member State', u'name': u'Kuwait..."
2,"{u'type': u'MultiPolygon', u'geodesic': True, ...",Feature,4,"{u'status': u'Member State', u'name': u'Algeri..."
3,"{u'type': u'MultiPolygon', u'geodesic': True, ...",Feature,5,"{u'status': u'Member State', u'name': u'Sudan'..."
4,"{u'type': u'Polygon', u'geodesic': True, u'coo...",Feature,6,"{u'status': u'Sovereignty unsettled', u'name':..."
5,"{u'type': u'Polygon', u'geodesic': True, u'coo...",Feature,9,"{u'status': u'Member State', u'name': u'Lesoth..."
6,"{u'type': u'MultiPolygon', u'geodesic': True, ...",Feature,10,"{u'status': u'Member State', u'name': u'Angola..."
7,"{u'type': u'MultiPolygon', u'geodesic': True, ...",Feature,11,"{u'status': u'Member State', u'name': u'Egypt'..."
8,"{u'type': u'Polygon', u'geodesic': True, u'coo...",Feature,12,"{u'status': u'Member State', u'name': u'Liberi..."
9,"{u'type': u'MultiPolygon', u'geodesic': True, ...",Feature,13,"{u'status': u'Member State', u'name': u'Libya'..."


proprieta = means_df['properties'] #.head()
print(type(proprieta))

In [198]:
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
  
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []
     
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)
      
    df = GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    df['geometry'] = map(lambda s: shape(s), df.geometry)   
    return df
# End fc2df

In [200]:
#paesi = fc2df(countries)

In [185]:
splittato = pd.Series(lambda x: pd.Series(x.split(',')))
splittato

0    <function <lambda> at 0x9fa6db54>
dtype: object

In [193]:
means_df.properties

0     {u'status': u'Member State', u'name': u'Zambia...
1     {u'status': u'Member State', u'name': u'Zimbab...
2     {u'status': u'Member State', u'name': u'Kuwait...
3     {u'status': u'Member State', u'name': u'Algeri...
4     {u'status': u'Member State', u'name': u'Sudan'...
5     {u'status': u'Sovereignty unsettled', u'name':...
6     {u'status': u'Member State', u'name': u'Lesoth...
7     {u'status': u'Member State', u'name': u'Angola...
8     {u'status': u'Member State', u'name': u'Egypt'...
9     {u'status': u'Member State', u'name': u'Liberi...
10    {u'status': u'Member State', u'name': u'Libya'...
11    {u'status': u'Sovereignty unsettled', u'name':...
12    {u'status': u'Sovereignty unsettled', u'name':...
13    {u'status': u'Sovereignty unsettled', u'name':...
14    {u'status': u'Member State', u'name': u'Madaga...
15    {u'status': u'Member State', u'name': u'Malawi...
16    {u'status': u'Member State', u'name': u'Mali',...
17    {u'status': u'Member State', u'name': u'Ba